In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import rasterio 
import matplotlib.pyplot as plt
from rasterio.plot import show

# Load input data
## Administrative boundaries

In [20]:
# load administrative boundaries

# Vitacura
#boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/boundary_CHL_vitacura_mapped_geom.geojson')
# Makati
#boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/PHL_makati.geojson')
# Hobart
boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/boundary_AUS_hobart_mapped.geojson')

# add centroid coordinates
boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
boundaries_cities["centroid_y"] = boundaries_cities.centroid.y

C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_20200/3291106954.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_20200/3291106954.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_y"] = boundaries_cities.centroid.y


In [21]:
boundaries_cities.head()

,country_iso3,city_id,city_name,boundary_data_source,geometry,centroid_x,centroid_y
0,AUS,AUS-Hobart,Hobart,osm,"MULTIPOLYGON (((147.31690 -42.87864, 147.31711...",147.327188,-42.882137


## Amenities

In [22]:
# get amenity data

# Vitacura
#amenity_dataportal_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/amenity/amenity_Vitacura.geojson')
# Makati
#amenity_dataportal_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/amenity/amenity_Makati.geojson')
# Hobart
amenity_dataportal_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/amenity/amenity_Hobart.geojson')


amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry
0,amenity,fountain,301928929,-42.878042,147.331027,Hobart,2021-10-06,amenity,dataportal,POINT (147.33103 -42.87804)
1,amenity,pub,303688146,-42.886843,147.331885,Hobart,2021-10-06,amenity,dataportal,POINT (147.33188 -42.88684)
2,amenity,pub,303688228,-42.890979,147.335032,Hobart,2021-10-06,amenity,dataportal,POINT (147.33503 -42.89098)
3,amenity,cinema,303743971,-42.885777,147.323891,Hobart,2021-10-06,amenity,dataportal,POINT (147.32389 -42.88578)
4,amenity,pub,319713233,-42.889523,147.323570,Hobart,2021-10-06,amenity,dataportal,POINT (147.32357 -42.88952)


In [23]:
# get amenity sectors
amenity_sectors=pd.read_csv("https://storage.googleapis.com/data_portal_exposure/data/amenity/param/osm_amenity_sectors_gcom.csv")
amenity_sectors.head()

,sector_name,gcom_sector_name,feature_key,feature_value,feature_category
0,Commercial,Commercial,building,\tretail,Commercial
1,Commercial,Commercial,building,warehouse,Commercial
2,Commercial,Commercial,building,commercial,Commercial
3,Commercial,Commercial,building,industrial,Commercial
4,Commercial,Commercial,amenity,marketplace,Others


In [24]:
# join amenity loaction with sector names
amenity_dataportal_cities = amenity_dataportal_cities.merge(amenity_sectors, 
                                     left_on='featureType',
                                     right_on='feature_value',
                                     how = "left")

In [25]:
# replace nan
amenity_dataportal_cities['sector_name'] = amenity_dataportal_cities['sector_name'].replace(np.nan, 'Other')
amenity_dataportal_cities['gcom_sector_name'] = amenity_dataportal_cities['gcom_sector_name'].replace(np.nan, 'Other')

In [26]:
amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry,sector_name,gcom_sector_name,feature_key,feature_value,feature_category
0,amenity,fountain,301928929,-42.878042,147.331027,Hobart,2021-10-06,amenity,dataportal,POINT (147.33103 -42.87804),Entertainment_Art_Culture,Society/community & culture,amenity,fountain,Entertainment_Art_Culture
1,amenity,pub,303688146,-42.886843,147.331885,Hobart,2021-10-06,amenity,dataportal,POINT (147.33188 -42.88684),Other,Other,NaN,NaN,NaN
2,amenity,pub,303688228,-42.890979,147.335032,Hobart,2021-10-06,amenity,dataportal,POINT (147.33503 -42.89098),Other,Other,NaN,NaN,NaN
3,amenity,cinema,303743971,-42.885777,147.323891,Hobart,2021-10-06,amenity,dataportal,POINT (147.32389 -42.88578),Entertainment_Art_Culture,Society/community & culture,amenity,cinema,Entertainment_Art_Culture
4,amenity,pub,319713233,-42.889523,147.323570,Hobart,2021-10-06,amenity,dataportal,POINT (147.32357 -42.88952),Other,Other,NaN,NaN,NaN


## Land Surface temperature

In [27]:
# Load LST

# seelect city id
city_id = "CHL-Vitacura"
city_id = "PHL-Makati"
city_id = "AUS-Hobart"

# define path
fpath = "https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_"+city_id+".tif"
# open raster data
city_lst_mean_r = rasterio.open(fpath)
# print infos
print(city_lst_mean_r.profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 97, 'height': 70, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 147.31544209498765,
       0.0, -0.00026949458523585647, -42.872815586831464), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [28]:
# read raster data for the selected city
city_lst_mean = city_lst_mean_r.read(1)
print(city_lst_mean)
print(city_lst_mean.mean())

[[35.64167089 35.75192478 35.90190819 ... 21.46251842 21.40975357
  21.40964475]
 [35.55497988 35.63689328 35.78625486 ... 21.50511809 21.44223511
  21.44212597]
 [35.59063739 35.59333342 35.74338708 ... 21.54364989 21.4720791
  21.47196938]
 ...
 [33.18948207 33.1891973  32.33414343 ... 19.7166899  19.7165383
  19.63614962]
 [32.78753211 32.7872519  31.86856532 ... 19.69931021 19.69915871
  19.61604407]
 [32.61790901 32.61762626 32.08000407 ... 19.67911934 19.67896794
  19.59184532]]
nan


# Compute amenity exposure

In [29]:
amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry,sector_name,gcom_sector_name,feature_key,feature_value,feature_category
0,amenity,fountain,301928929,-42.878042,147.331027,Hobart,2021-10-06,amenity,dataportal,POINT (147.33103 -42.87804),Entertainment_Art_Culture,Society/community & culture,amenity,fountain,Entertainment_Art_Culture
1,amenity,pub,303688146,-42.886843,147.331885,Hobart,2021-10-06,amenity,dataportal,POINT (147.33188 -42.88684),Other,Other,NaN,NaN,NaN
2,amenity,pub,303688228,-42.890979,147.335032,Hobart,2021-10-06,amenity,dataportal,POINT (147.33503 -42.89098),Other,Other,NaN,NaN,NaN
3,amenity,cinema,303743971,-42.885777,147.323891,Hobart,2021-10-06,amenity,dataportal,POINT (147.32389 -42.88578),Entertainment_Art_Culture,Society/community & culture,amenity,cinema,Entertainment_Art_Culture
4,amenity,pub,319713233,-42.889523,147.323570,Hobart,2021-10-06,amenity,dataportal,POINT (147.32357 -42.88952),Other,Other,NaN,NaN,NaN


In [30]:
# join amenity loaction with administrative boundaries data to get city_id filed
amenity_dataportal_cities = amenity_dataportal_cities.merge(boundaries_cities[["country_iso3", "city_id","city_name"]], 
                                     left_on='cityName',
                                     right_on='city_name',
                                     how = "left")

In [31]:
# create empty dataframe
amenity_exposure = pd.DataFrame(columns=['city_name',
                                         'city_lst_avg',
                                         'city_lst_perc_90',
                                         'feature_key',
                                         'feature_value',
                                         'feature_category',
                                         'latitude',
                                         'longitude',
                                         'geometry',
                                         'country_iso',
                                         'city_id',
                                         'sector_name',
                                         'gcom_sector_name',
                                         'exposure_lst_mean',
                                         'lst_value_dev_city_mean',
                                         'lst_pecent_dev_city_mean',
                                         'lst_dev_city_mean_exposure_class',
                                         'lst_value_dev_city_perc_90',
                                         'lst_pecent_dev_city_perc_90',
                                         'lst_dev_city_perc90_exposure_class'
                                         ])

In [32]:

for i in range(len(boundaries_cities)):
    city_id = boundaries_cities.loc[i, 'city_id']
    print("\n City Id: "+city_id)
    # select amenity data for the selectd city
    city_amenity =  amenity_dataportal_cities[amenity_dataportal_cities.city_id == city_id]
    # select LST data
    fpath = "https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_"+city_id+".tif"
    print("\n path: "+fpath)
    city_lst_mean_r = rasterio.open(fpath)
    print(city_lst_mean_r)
    # get amenity coordinates
    city_amenity_coords = [(x,y) for x, y in zip(city_amenity.longitude, city_amenity.latitude)]
    # extract lst values for amenity coordinates
    city_amenity['exposure_lst_mean'] = [x[0] for x in city_lst_mean_r.sample(city_amenity_coords)]
    # get average lst value within all city raster area
    city_lst_mean_r_band1= city_lst_mean_r.read(1)
    city_lst_mean_r_mean = np.nanmean(city_lst_mean_r_band1)
    city_amenity['city_lst_avg'] = city_lst_mean_r_mean
    # get 90th percentile value within all city raster area
    city_lst_mean_r_perc_90 = np.nanpercentile(city_lst_mean_r_band1, 90)
    city_amenity['city_lst_perc_90'] = city_lst_mean_r_perc_90
    # compute mean-based indicators
    city_amenity['lst_value_dev_city_mean'] = round(city_amenity['exposure_lst_mean'] - city_amenity['city_lst_avg'],2)
    city_amenity['lst_pecent_dev_city_mean'] = round((city_amenity['exposure_lst_mean'] * 100)/city_amenity['city_lst_avg'],2)
    city_amenity.loc[city_amenity['lst_value_dev_city_mean'] > 0, 'lst_dev_city_mean_exposure_class'] = 'Exposed'
    city_amenity.loc[city_amenity['lst_value_dev_city_mean'] < 0, 'lst_dev_city_mean_exposure_class'] = 'Not Exposed'
    # compute 90 percentile-based indicators
    city_amenity['lst_value_dev_city_perc_90'] = round(city_amenity['exposure_lst_mean'] - city_amenity['city_lst_perc_90'],2)
    city_amenity['lst_pecent_dev_city_perc_90'] = round((city_amenity['exposure_lst_mean'] * 100)/city_amenity['city_lst_perc_90'],2)
    city_amenity.loc[city_amenity['lst_value_dev_city_perc_90'] > 0, 'lst_dev_city_perc90_exposure_class'] = 'Exposed'
    city_amenity.loc[city_amenity['lst_value_dev_city_perc_90'] < 0, 'lst_dev_city_perc90_exposure_class'] = 'Not Exposed'
    # append with intiailized dataframe
    amenity_exposure = amenity_exposure.append(city_amenity, ignore_index=True)
    


 City Id: AUS-Hobart

 path: https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_AUS-Hobart.tif
<open DatasetReader name='https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_AUS-Hobart.tif' mode='r'>


In [33]:
# convert into csv
amenity_exposure_csv = amenity_exposure[['city_name',
                                         'city_lst_avg',
                                         'city_lst_perc_90',
                                         'feature_key',
                                         'feature_value',
                                         'feature_category',
                                         'latitude',
                                         'longitude',
                                         'geometry',
                                         'country_iso',
                                         'city_id',
                                         'sector_name',
                                         'gcom_sector_name',
                                         'exposure_lst_mean',
                                         'lst_value_dev_city_mean',
                                         'lst_pecent_dev_city_mean',
                                         'lst_dev_city_mean_exposure_class',
                                         'lst_value_dev_city_perc_90',
                                         'lst_pecent_dev_city_perc_90',
                                         'lst_dev_city_perc90_exposure_class'
                                         ]].to_csv()

In [34]:
amenity_exposure.head()

,city_name,city_lst_avg,city_lst_perc_90,feature_key,feature_value,feature_category,latitude,longitude,geometry,country_iso,...,lst_pecent_dev_city_perc_90,lst_dev_city_perc90_exposure_class,featureCategory,featureType,id,cityName,integrationDate,objectType,projectName,country_iso3
0,Hobart,32.126292,36.642406,amenity,fountain,Entertainment_Art_Culture,-42.878042,147.331027,POINT (147.33103 -42.87804),NaN,...,90.69,Not Exposed,amenity,fountain,301928929.0,Hobart,2021-10-06,amenity,dataportal,AUS
1,Hobart,32.126292,36.642406,NaN,NaN,NaN,-42.886843,147.331885,POINT (147.33188 -42.88684),NaN,...,91.27,Not Exposed,amenity,pub,303688146.0,Hobart,2021-10-06,amenity,dataportal,AUS
2,Hobart,32.126292,36.642406,NaN,NaN,NaN,-42.890979,147.335032,POINT (147.33503 -42.89098),NaN,...,96.81,Not Exposed,amenity,pub,303688228.0,Hobart,2021-10-06,amenity,dataportal,AUS
3,Hobart,32.126292,36.642406,amenity,cinema,Entertainment_Art_Culture,-42.885777,147.323891,POINT (147.32389 -42.88578),NaN,...,84.81,Not Exposed,amenity,cinema,303743971.0,Hobart,2021-10-06,amenity,dataportal,AUS
4,Hobart,32.126292,36.642406,NaN,NaN,NaN,-42.889523,147.323570,POINT (147.32357 -42.88952),NaN,...,92.09,Not Exposed,amenity,pub,319713233.0,Hobart,2021-10-06,amenity,dataportal,AUS


In [36]:
amenity_exposure.to_csv('amenity_exposure_heat_hobart.csv')

# Plot

In [18]:
#######################################################################################################################
#  plot amenity within one city  
#######################################################################################################################

# select city name
city_name = "Vitacura"
# select amenity data for the selectd city
city_amenity =  amenity_dataportal_cities[amenity_dataportal_cities.cityName == city_name]
# select boundaries data for the selected city
city_boundaries =  boundaries_cities[boundaries_cities.city_name == city_name]

# color definition in boundary polygon
style_function_Boundaries = lambda x: {
    'fillColor':"gray",
    'color': "black",
    'weight': 3.5,
    'fillOpacity': 0.3
}


# Plot map background
m = folium.Map(location=[float(city_boundaries["centroid_y"]), float(city_boundaries["centroid_x"])],
               zoom_start = 8)


# plot administrative boundaries of selected city
folium.GeoJson(
    city_boundaries,
    name = "Administrative boundaries",
    style_function=style_function_Boundaries,
).add_to(m)

# plot amenities of the selected city
for i in range(0,len(city_amenity)):
    folium.CircleMarker(
        location=[city_amenity.iloc[i]['latitude'], city_amenity.iloc[i]['longitude']],
        popup=city_amenity.iloc[i]['featureType'],
        radius=2,
        color="red",
        fill_color="red",
        fill=True,
        name = "amenities",
    ).add_to(m)

folium.LayerControl().add_to(m)

m

TypeError: cannot convert the series to <class 'float'>